In [0]:
import json
import os
import sys
import tensorflow as tf

In [2]:
if 'google.colab' in sys.modules:
    !git clone https://github.com/leighlin0511/ll_utils
    sys.path.append('/content/ll_utils')
    
    from google.colab import auth
    auth.authenticate_user()    

import ll_cellular.mail_ll as mail_ll

Cloning into 'rxrx1-utils'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 99 (delta 48), reused 92 (delta 42), pack-reused 0
Unpacking objects: 100% (99/99), done.


## Train

Set `MODEL_DIR` to be a Google Cloud Storage bucket that you can write to.   The code will write your checkpoins to this directory.

In [1]:
MODEL_DIR = 'gs://kaggle-recursive-cell-image-storage/saved_model/1562302335'
URL_BASE_PATH = 'gs://rxrx1-us-central1/tfrecords/random-42'

# make sure we're in a TPU runtime
assert 'COLAB_TPU_ADDR' in os.environ

# set TPU-relevant args
tpu_grpc = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
num_shards = 8  # colab uses Cloud TPU v2-8

# upload credentials to the TPU
with tf.Session(tpu_grpc) as sess:
    data = json.load(open('/content/adc.json'))
    tf.contrib.cloud.configure_gcs(sess, credentials=data)

tf.logging.set_verbosity(tf.logging.INFO)

main_ll(use_tpu=True,
     tpu=tpu_grpc,
     gcp_project=None,
     tpu_zone=None,
     url_base_path=URL_BASE_PATH,
     use_cache=False,
     model_dir=MODEL_DIR,
     train_epochs=1,
     train_batch_size=512,
     num_train_images=73030,
     epochs_per_loop=1,
     log_step_count_epochs=1,
     num_cores=num_shards,
     data_format='channels_last',
     transpose_input=True,
     tf_precision='bfloat16',
     n_classes=1108,
     momentum=0.9,
     weight_decay=1e-4,
     base_learning_rate=0.2,
     warmup_epochs=5)

NameError: name 'os' is not defined